In [1]:
import os
import django
from pathlib import Path
os.chdir(Path.cwd().parent)
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'testproject.settings')
django.setup()

RecursionError: maximum recursion depth exceeded in comparison

# Formkit-Ninja

What is Formkit-Ninja?
It's a Django application to store and convert FormKit schemas between JSON, Pydantic models, and database backend. It supports most of the FormKit syntax as well as some of the "extra" goodies.

In [2]:

# Here we're defining some functions to help us display data in a nice way 

from typing import Any
from rich.table import Table
from rich.jupyter import print
from pydantic import BaseModel

def kvtable(inputelement: dict[str, Any], title: str | None = None):
    table = Table(title=title)
    table.add_column("Key", style="cyan", no_wrap=True)
    table.add_column("Value", style="magenta")
    for key, value in sorted(inputelement.items()):
        table.add_row(str(key), str(value))
    print(table)

def modeltable(inputmodel: BaseModel):
    kvtable(inputmodel.model_dump(exclude_none=True, by_alias=True), title=inputmodel.__class__.__name__)

def manytable(inputelements: list[dict[str, Any]], title: str | None = None):
    table = Table(title=title)
    table.add_column("Index", style="cyan", no_wrap=True)
    # Set of all keys
    keys = set()
    for element in inputelements:
        keys.update(element.keys())
    # These are the "rows"
    rows = sorted(keys)
    # These are the "columns"
    range(len(inputelements))
    # Add the columns
    for index, i in enumerate(inputelements):
        table.add_column(str(index), style="magenta")
    # Add the rows
    for row in rows:
        content = []
        for index, i in enumerate(inputelements):
            content.append(str(i.get(row, "")))
        table.add_row(row, *content)
    print(table)

def manynodetable(inputelements: list[BaseModel], title: str | None = None):
    manytable([i.model_dump(exclude_none=True, by_alias=True) for i in inputelements], title=title) 
    


In [3]:
# Add a "Telephone" input

# This is a simple input with validation and a label
# Note that the label is translated using the gettext function (client side)
tel = {
    "$formkit": "tel",
    "label": '$gettext("Phone number")',
    "maxLength": 8,
    "name": "phone_number",
    "validation": "number|length:8,8",
}
kvtable(tel)

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key        ┃ Value                    ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit   │ tel                      │
│ label      │ $gettext("Phone number") │
│ maxLength  │ 8                        │
│ name       │ phone_number             │
│ validation │ number|length:8,8        │
└────────────┴──────────────────────────┘

In [4]:

# This is in Python / Pydantic a "TelNode"
# We can create this node from the dictionary above

from formkit_ninja.formkit_schema import TelNode

node = TelNode(**tel)

modeltable(node)

                 TelNode                 
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key        ┃ Value                    ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit   │ tel                      │
│ label      │ $gettext("Phone number") │
│ maxLength  │ 8                        │
│ name       │ phone_number             │
│ validation │ number|length:8,8        │
└────────────┴──────────────────────────┘

In [5]:
# What if we don't know the type of node beforehand?
# We can use the "FormkitNode" class to create the node from the dictionary
from formkit_ninja.formkit_schema import DiscriminatedNodeType
node = DiscriminatedNodeType(**tel)

# The DiscriminatedNodeType can convert a variety of "node" types
modeltable(node)

          DiscriminatedNodeType          
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key        ┃ Value                    ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit   │ tel                      │
│ label      │ $gettext("Phone number") │
│ maxLength  │ 8                        │
│ name       │ phone_number             │
│ validation │ number|length:8,8        │
└────────────┴──────────────────────────┘

In [6]:
node

DiscriminatedNodeType(root=TelNode(children=None, key=None, if_condition=None, for_loop=None, bind=None, meta=None, id=None, name='phone_number', label='$gettext("Phone number")', help=None, validation='number|length:8,8', validationLabel=None, validationVisibility=None, validationMessages=None, placeholder=None, value=None, prefixIcon=None, classes=None, additional_props={'maxLength': 8}, node_type='formkit', readonly=None, formkit='tel'))

In [7]:
# And, we can save this in the database...

from formkit_ninja.models import FormKitSchemaNode

# This has a `from_pydantic` method
nodes = list(FormKitSchemaNode.from_pydantic(node))


[03:34:32] Creating  9bc164cb-12e1-45c4-8dfd-2abc34cf3e79                                             ]8;id=158133;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=163272;file:///app/src/formkit_ninja/models.py#571\571]8;;\

           Yielding: Node: phone_number                                                               ]8;id=946727;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=822911;file:///app/src/formkit_ninja/models.py#593\593]8;;\

In [8]:
nodes

[<FormKitSchemaNode: Node: phone_number>]

In [9]:
for n in nodes:
    kvtable(n.get_node_values())
    modeltable(n.get_node())

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key              ┃ Value                    ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit         │ tel                      │
│ additional_props │ {'maxLength': 8}         │
│ label            │ $gettext("Phone number") │
│ name             │ phone_number             │
│ validation       │ number|length:8,8        │
└──────────────────┴──────────────────────────┘

                 TelNode                 
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key        ┃ Value                    ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit   │ tel                      │
│ label      │ $gettext("Phone number") │
│ maxLength  │ 8                        │
│ name       │ phone_number             │
│ validation │ number|length:8,8        │
└────────────┴──────────────────────────┘

## A More Complex Example

The following example is a "datepicker"

In [10]:
sf_41_datpicker = {
    "$formkit": "datepicker",
    "_currentDate": "$getCurrentDate",
    "calendarIcon": "calendar",
    "format": "DD/MM/YYYY",
    "id": "date",
    "key": "date",
    "label": '$gettext("Date")',
    "name": "date",
    "nextIcon": "angleRight",
    "prevIcon": "angleLeft",
    "sectionsSchema": {
        "day": {
            "children": [
                "$day.getDate()",
                {
                    "children": [
                        {
                            "children": [
                                {
                                    "$el": "div",
                                    "attrs": {"class": "formkit-day-highlight"},
                                    "if": "$attrs._currentDate().year === $day.getFullYear()",
                                }
                            ],
                            "if": "$attrs._currentDate().month === $day.getMonth()",
                        }
                    ],
                    "if": "$attrs._currentDate().day === $day.getDate()",
                },
            ]
        }
    },
}


In [11]:
# When we run this we should get a valid node
sf_41_node = DiscriminatedNodeType(**sf_41_datpicker)

In [12]:
kvtable(sf_41_datpicker)
modeltable(sf_41_node)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key            ┃ Value                                                                                          ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit       │ datepicker                                                                                     │
│ _currentDate   │ $getCurrentDate                                                                                │
│ calendarIcon   │ calendar                                                                                       │
│ format         │ DD/MM/YYYY                                                                                     │
│ id             │ date                                                                                           │
│ key            │ date                                                                                           │
│ label          │ $gettext("Date")                                                                               │
│ name           │ date                                                                                           │
│ nextIcon       │ angleRight                                                                                     │
│ prevIcon       │ angleLeft                                                                                      │
│ sectionsSchema │ {'day': {'children': ['$day.getDate()', {'children': [{'children': [{'$el': 'div', 'attrs':    │
│                │ {'class': 'formkit-day-highlight'}, 'if': '$attrs._currentDate().year ===                      │
│                │ $day.getFullYear()'}], 'if': '$attrs._currentDate().month === $day.getMonth()'}], 'if':        │
│                │ '$attrs._currentDate().day === $day.getDate()'}]}}                                             │
└────────────────┴────────────────────────────────────────────────────────────────────────────────────────────────┘

                                               DiscriminatedNodeType                                               
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key            ┃ Value                                                                                          ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit       │ datepicker                                                                                     │
│ _currentDate   │ $getCurrentDate                                                                                │
│ calendarIcon   │ calendar                                                                                       │
│ format         │ DD/MM/YYYY                                                                                     │
│ id             │ date                                                                                           │
│ key            │ date                                                                                           │
│ label          │ $gettext("Date")                                                                               │
│ name           │ date                                                                                           │
│ nextIcon       │ angleRight                                                                                     │
│ prevIcon       │ angleLeft                                                                                      │
│ sectionsSchema │ {'day': {'children': ['$day.getDate()', {'children': [{'children': [{'$el': 'div', 'attrs':    │
│                │ {'class': 'formkit-day-highlight'}, 'if': '$attrs._currentDate().year ===                      │
│                │ $day.getFullYear()'}], 'if': '$attrs._currentDate().month === $day.getMonth()'}], 'if':        │
│                │ '$attrs._currentDate().day === $day.getDate()'}]}}                                             │
└────────────────┴────────────────────────────────────────────────────────────────────────────────────────────────┘

In [13]:
# Does it serialize to postgres? Let's see!
# from_pydantic will return a list of nodes
nodes = list(FormKitSchemaNode.from_pydantic(sf_41_node))




[03:34:33] Creating  876cd51c-fe51-4f80-a54b-822c3c23aa73                                             ]8;id=362397;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=556779;file:///app/src/formkit_ninja/models.py#571\571]8;;\

           Yielding: Node: date                                                                       ]8;id=714230;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=669302;file:///app/src/formkit_ninja/models.py#593\593]8;;\

In [14]:
for n in nodes:
    kvtable(n.get_node_values())
    modeltable(n.get_node())

# Note that there is one difference on postgres
# Because `sectionsSchema` is something we don't
# have in the postgres schema, it's held within the `additional_props` field

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key              ┃ Value                                                                                        ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit         │ datepicker                                                                                   │
│ additional_props │ {'_currentDate': '$getCurrentDate', 'sectionsSchema': {'day': {'children':                   │
│                  │ ['$day.getDate()', {'children': [{'children': [{'$el': 'div', 'attrs': {'class':             │
│                  │ 'formkit-day-highlight'}, 'if': '$attrs._currentDate().year === $day.getFullYear()'}], 'if': │
│                  │ '$attrs._currentDate().month === $day.getMonth()'}], 'if': '$attrs._currentDate().day ===    │
│                  │ $day.getDate()'}]}}}                                                                         │
│ calendarIcon     │ calendar                                                                                     │
│ format           │ DD/MM/YYYY                                                                                   │
│ id               │ date                                                                                         │
│ key              │ date                                                                                         │
│ label            │ $gettext("Date")                                                                             │
│ name             │ date                                                                                         │
│ nextIcon         │ angleRight                                                                                   │
│ prevIcon         │ angleLeft                                                                                    │
└──────────────────┴──────────────────────────────────────────────────────────────────────────────────────────────┘

                                                  DatePickerNode                                                   
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key            ┃ Value                                                                                          ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit       │ datepicker                                                                                     │
│ _currentDate   │ $getCurrentDate                                                                                │
│ calendarIcon   │ calendar                                                                                       │
│ format         │ DD/MM/YYYY                                                                                     │
│ id             │ date                                                                                           │
│ key            │ date                                                                                           │
│ label          │ $gettext("Date")                                                                               │
│ name           │ date                                                                                           │
│ nextIcon       │ angleRight                                                                                     │
│ prevIcon       │ angleLeft                                                                                      │
│ sectionsSchema │ {'day': {'children': ['$day.getDate()', {'children': [{'children': [{'$el': 'div', 'attrs':    │
│                │ {'class': 'formkit-day-highlight'}, 'if': '$attrs._currentDate().year ===                      │
│                │ $day.getFullYear()'}], 'if': '$attrs._currentDate().month === $day.getMonth()'}], 'if':        │
│                │ '$attrs._currentDate().day === $day.getDate()'}]}}                                             │
└────────────────┴────────────────────────────────────────────────────────────────────────────────────────────────┘

In [15]:
# Schemas
# Usually one input is not enough, we need a schema to hold multiple inputs
# This is a JSON object not real javascript
schema = '''
[
  {
    $el: 'h1',
    children: 'Register',
    attrs: {
      class: 'text-2xl font-bold mb-4',
    },
  },
  {
    $formkit: 'text',
    name: 'email',
    label: 'Email',
    help: 'This will be used for your account.',
    validation: 'required|email',
  },
  {
    $formkit: 'password',
    name: 'password',
    label: 'Password',
    help: 'Enter your new password.',
    validation: 'required|length:5,16',
  },
  {
    $formkit: 'password',
    name: 'password_confirm',
    label: 'Confirm password',
    help: 'Enter your new password again to confirm it.',
    validation: 'required|confirm',
    validationLabel: 'password confirmation',
  },
  {
    $cmp: 'FormKit',
    props: {
      name: 'eu_citizen',
      type: 'checkbox',
      id: 'eu',
      label: 'Are you a european citizen?',
    },
  },
  {
    $formkit: 'select',
    if: '$get(eu).value', // 👀 Oooo, conditionals!
    name: 'cookie_notice',
    label: 'Cookie notice frequency',
    options: {
      refresh: 'Every page load',
      hourly: 'Ever hour',
      daily: 'Every day',
    },
    help: 'How often should we display a cookie notice?',
  },
]
'''

import json5
schema_as_js = json5.loads(schema)

In [16]:
# So, what happens when we try to parse this?
# It's a list of FormKitNodes
for n in schema_as_js:
    kvtable(n)

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key      ┃ Value                                ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $el      │ h1                                   │
│ attrs    │ {'class': 'text-2xl font-bold mb-4'} │
│ children │ Register                             │
└──────────┴──────────────────────────────────────┘

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key        ┃ Value                               ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit   │ text                                │
│ help       │ This will be used for your account. │
│ label      │ Email                               │
│ name       │ email                               │
│ validation │ required|email                      │
└────────────┴─────────────────────────────────────┘

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key        ┃ Value                    ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit   │ password                 │
│ help       │ Enter your new password. │
│ label      │ Password                 │
│ name       │ password                 │
│ validation │ required|length:5,16     │
└────────────┴──────────────────────────┘

┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key             ┃ Value                                        ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit        │ password                                     │
│ help            │ Enter your new password again to confirm it. │
│ label           │ Confirm password                             │
│ name            │ password_confirm                             │
│ validation      │ required|confirm                             │
│ validationLabel │ password confirmation                        │
└─────────────────┴──────────────────────────────────────────────┘

┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key   ┃ Value                                                                                          ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $cmp  │ FormKit                                                                                        │
│ props │ {'name': 'eu_citizen', 'type': 'checkbox', 'id': 'eu', 'label': 'Are you a european citizen?'} │
└───────┴────────────────────────────────────────────────────────────────────────────────────────────────┘

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key      ┃ Value                                                                       ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ $formkit │ select                                                                      │
│ help     │ How often should we display a cookie notice?                                │
│ if       │ $get(eu).value                                                              │
│ label    │ Cookie notice frequency                                                     │
│ name     │ cookie_notice                                                               │
│ options  │ {'refresh': 'Every page load', 'hourly': 'Ever hour', 'daily': 'Every day'} │
└──────────┴─────────────────────────────────────────────────────────────────────────────┘

In [17]:
from formkit_ninja.formkit_schema import DiscriminatedNodeTypeSchema
from formkit_ninja.models import FormKitSchema

# Our "schema"

schema_as_pydantic = DiscriminatedNodeTypeSchema(schema_as_js)



nodes = FormKitSchema.from_pydantic(schema_as_pydantic)
print(nodes)

           Creating  d9933d07-4216-4cb9-8b6e-8e50edd830a2                                             ]8;id=571736;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=778473;file:///app/src/formkit_ninja/models.py#571\571]8;;\

           Yielding: $el d9933d07-4216-4cb9-8b6e-8e50edd830a2                                         ]8;id=419870;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=521789;file:///app/src/formkit_ninja/models.py#593\593]8;;\

               Node: Register                                                                         ]8;id=328986;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=535476;file:///app/src/formkit_ninja/models.py#636\636]8;;\

           Saving $el d9933d07-4216-4cb9-8b6e-8e50edd830a2                                            ]8;id=843172;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=812932;file:///app/src/formkit_ninja/models.py#765\765]8;;\

           Creating  5207ecdd-757c-49ad-923d-f3929d908bf6                                             ]8;id=709744;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=664667;file:///app/src/formkit_ninja/models.py#571\571]8;;\

...

           Saving $cmp 8c4c2567-0293-4072-99a4-5600c9770fdc                                           ]8;id=871595;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=298622;file:///app/src/formkit_ninja/models.py#765\765]8;;\

           Creating  bcf095a2-dd18-4999-9b28-cc454598356f                                             ]8;id=170266;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=254117;file:///app/src/formkit_ninja/models.py#571\571]8;;\

           Yielding: Node: cookie_notice                                                              ]8;id=22832;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=978479;file:///app/src/formkit_ninja/models.py#593\593]8;;\

           Saving Node: cookie_notice                                                                 ]8;id=697387;file:///app/src/formkit_ninja/models.py\models.py]8;;\:]8;id=557222;file:///app/src/formkit_ninja/models.py#765\765]8;;\

fb684b7f-b1d7-4ec3-af0e-ec32a511e57a

In [18]:
nodes.publish()

<PublishedForm: fb684b7f-b1d7-4ec3-af0e-ec32a511e57a (published: 2025-04-04 - current)>

In [19]:
nodes

<FormKitSchema: fb684b7f-b1d7-4ec3-af0e-ec32a511e57a>

In [20]:
manytable(schema_as_js)
manytable([n.get_node_values() for n in nodes])
manynodetable([n.get_node() for n in nodes])

┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Index           ┃ 0             ┃ 1             ┃ 2             ┃ 3             ┃ 4             ┃ 5             ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ $cmp            │               │               │               │               │ FormKit       │               │
│ $el             │ h1            │               │               │               │               │               │
│ $formkit        │               │ text          │ password      │ password      │               │ select        │
│ attrs           │ {'class':     │               │               │               │               │               │
│                 │ 'text-2xl     │               │               │               │               │               │
│                 │ font-bold     │               │               │               │               │               │
│                 │ mb-4'}        │               │               │               │               │               │
│ children        │ Register      │               │               │               │               │               │
│ help            │               │ This will be  │ Enter your    │ Enter your    │               │ How often     │
│                 │               │ used for your │ new password. │ new password  │               │ should we     │
│                 │               │ account.      │               │ again to      │               │ display a     │
│                 │               │               │               │ confirm it.   │               │ cookie        │
│                 │               │               │               │               │               │ notice?       │
│ if              │               │               │               │               │               │ $get(eu).val… │
│ label           │               │ Email         │ Password      │ Confirm       │               │ Cookie notice │
│                 │               │               │               │ password      │               │ frequency     │
│ name            │               │ email         │ password      │ password_con… │               │ cookie_notice │
│ options         │               │               │               │               │               │ {'refresh':   │
│                 │               │               │               │               │               │ 'Every page   │
│                 │               │               │               │               │               │ load',        │
│                 │               │               │               │               │               │ 'hourly':     │
│                 │               │               │               │               │               │ 'Ever hour',  │
│                 │               │               │               │               │               │ 'daily':      │
│                 │               │               │               │               │               │ 'Every day'}  │
│ props           │               │               │               │               │ {'name':      │               │
│                 │               │               │               │               │ 'eu_citizen', │               │
│                 │               │               │               │               │ 'type':       │               │
│                 │               │               │               │               │ 'checkbox',   │               │
│                 │               │               │               │               │ 'id': 'eu',   │               │
│                 │               │               │               │               │ 'label': 'Are │               │
│                 │               │               │               │               │ you a         │               │
│                 │               │               │     

TypeError: 'FormKitSchema' object is not iterable

In [ ]:
# nodes[3].get_node_values()
schema_as_js[3]

from formkit_ninja.formkit_schema import PasswordNode
PasswordNode(**schema_as_js[3])

schema_as_js[3]["validation-label"] = schema_as_js[3].pop("validationLabel")
PasswordNode(**schema_as_js[3])

In [ ]:
sf_41_node

In [ ]:
schema